In [1]:
import numpy as np 
import pandas as pd
import datetime as dt
import openpyxl
import calendar
import xlsxwriter

In [4]:
raw_df=pd.read_excel('cd_nursingcalendar.xlsx', sheet_name="Hoja1")
#aw_df=raw_df.drop(['ID'], axis=1)
#raw_df=raw_df.head(15)

In [5]:
work_schedule_id=pd.read_excel('cd_nursingcalendar.xlsx', sheet_name="work_schedules")
#work_schedule_id=work_schedule_id.rename(columns=work_schedule_id.iloc[0])
#work_schedule_id=work_schedule_id.drop([0])
work_schedule_id=work_schedule_id.drop(['entrance_time','departure_time','hospitalid','floor_id'],axis=1)
#work_schedule_id=work_schedule_id[['name','WORK_SCHEDULEID']]

In [6]:
nurses=pd.read_excel('cd_nursingcalendar.xlsx', sheet_name="nurses")
nurses=nurses[['id','name']]

In [7]:
#DUPLICAR LAS PRIMERAS DOS SEMANAS PARA HACER 4 SEMANAS EN TOTAL
weeks_4=raw_df.iloc[:,4:18]
#DUPLICAR LA PRIMERA SEMANA PARA HACER 5 SEMANAS EN TOTAL
week_1=raw_df.iloc[:,4:11]
raw_df=raw_df.join(weeks_4,how='left',lsuffix='_left', rsuffix='_right')
raw_df=raw_df.join(week_1,how='left',lsuffix='_left', rsuffix='_right')

In [8]:
dias_semana= { 'Domingo': '0',
'Lunes':'1',
'Martes':'2', 
'Miercoles':'3',
'Jueves':'4',
'Viernes':'5',
'Sabado':'6'}

onlyDays=list(dias_semana.keys())

ind=[]
for i in range(0,5):
    for j in range(len(onlyDays)):
        a=str(onlyDays[j])+"{}".format(i)
        ind.append(a)
index=ind.copy()
index.insert(0,"dias descanso")
index.insert(0,"floor_name")
index.insert(0,"schedule_name")
index.insert(0,"name")

raw_df.columns=index

In [9]:
left_join= pd.merge(raw_df, work_schedule_id, on =['schedule_name','floor_name'], how ='left')
join_= pd.merge(left_join, nurses, on ='name',how ='left')

In [10]:
#join_=pd.melt(join_, id_vars =['name','WORK_SCHEDULEID','id'], value_vars =ind,var_name="Days",value_name='Work')
join_=pd.melt(join_, id_vars =['name','id_x','id_y'], value_vars =ind,var_name="Days",value_name='Work')
#join_=join_.sort_values(['name','Days'],ascending = [True,onlyDays])
join_=join_.reset_index(drop=True)

In [ ]:
dias_semana= {
'Domingo': '0',
'Lunes':'1',
'Martes':'2', 
'Miercoles':'3',
'Jueves':'4',
'Viernes':'5',
'Sabado':'6',
}

onlyDays=list(dias_semana.keys())

ind=[]
for i in range(0,5):
    for j in range(len(onlyDays)):
        a=str(onlyDays[j])+"{}".format(i)
        ind.append(a)

##############################################################

df_list = []

for dia in ind:
   df_list.append(join_[join_['Days']==dia ])

ordered_df = pd.concat(df_list)
ordered_df=ordered_df.reset_index(drop=True)


#############################################################

enfermeras=list(set(join_.name))
data=pd.DataFrame()
for nurse in enfermeras:
    df_y=ordered_df[ordered_df['name'] == nurse].reset_index(drop=True)
    date_today = pd.to_datetime('now').month
    hoy=dt.date.today()
    mes=hoy.month
    anio=hoy.year
    ultimoDia_mes=calendar.monthrange(anio, mes)[1]
    primerDia_mes=calendar.monthrange(anio, mes)[0]

    if primerDia_mes==0:
        day="Lunes0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    elif primerDia_mes==1:
        day="Martes0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    elif primerDia_mes==2:
        day="Miercoles0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    elif primerDia_mes==3:
        day="Jueves0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    elif primerDia_mes==4:
        day="Viernes0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    elif primerDia_mes==5:
        day="Sabado0"
        firstIndex=df_y.index[df_y['Days'] == day][0]
    else:
        day="Domingo0"
        firstIndex=df_y.index[df_y['Days'] == day][0]

    fechas=list(pd.period_range(start=f'{mes}'+'/1/'+f'{anio}', end=f'{mes}/'+f'{ultimoDia_mes}'+'/'+f'{anio}', freq='D'))
    df_y=df_y.iloc[firstIndex:,:]
    df_y=df_y.iloc[0:len(fechas),:]
    r,c =df_y.shape
    for i in range(r):
        df_y.iloc[i,3]=fechas[i]

    data=data.append(df_y, ignore_index = True)    

In [13]:
data=data[data['Work'].notna()].reset_index(drop=True)
data['Days'] = data['Days'].astype(str)
data['Days']= pd.to_datetime(data['Days'],format='%Y-%m-%d').dt.date

In [14]:
descanso=raw_df[['name','dias descanso']]
descanso=descanso.rename({'dias descanso':'dias_descanso'}, axis=1)
auxiliar=descanso[descanso['dias_descanso'].str.contains(",", na=False)]
auxiliar['day']=auxiliar.dias_descanso.str.split(",")
#auxiliar.loc[:,('day')]=auxiliar.dias_descanso.str.split(",")
descanso=descanso.rename({'dias_descanso':'day'}, axis=1)
descanso.update(auxiliar)
descanso = descanso.dropna()
enfermeras=list(set(descanso.name))

In [ ]:
dataframe=pd.DataFrame()
for nurse in enfermeras:
    DD=[]
    df_y=data[data['name'] == nurse]
    df_y['Days']=pd.to_datetime(df_y.Days)
    df_x=descanso[descanso['name'] == nurse]
    #lista donde guardará los dias formato fecha que una enfermera no trabaja
    mes= pd.to_datetime('now').month
    anio= pd.to_datetime('now').year
    if type(df_x.iloc[0,1])==list:
        for j in range(len(df_x.iloc[0,1])):
            dia=df_x.iloc[0,1][j]
            value=f'{anio}-'+f'{mes}'+f'-{dia}'
            DD.append(value)
        DD=pd.to_datetime(DD, errors='coerce',format='%Y-%m-%d')
        #print(DD)
        df_y=df_y[~df_y.Days.isin(DD)]
        dataframe=dataframe.append(df_y, ignore_index = True)
    else:
        dia=df_x.iloc[0,1]
        value=f'{anio}-'+f'{mes}'+f'-{dia}'
        DD.append(value)
        DD=pd.to_datetime(DD,format='%Y-%m-%d')
        #print(DD)
        df_y=df_y[~df_y.Days.isin(DD)]
        dataframe=dataframe.append(df_y, ignore_index = True)


In [16]:
descanso=raw_df[['name','dias descanso']]
descanso=descanso.rename({'dias descanso':'dias_descanso'}, axis=1)
auxiliar=descanso[descanso['dias_descanso'].str.contains(",", na=False)]
auxiliar['day']=auxiliar.dias_descanso.str.split(",")
descanso=descanso.rename({'dias_descanso':'day'}, axis=1)
descanso.update(auxiliar)
descanso = pd.DataFrame(descanso[descanso.day.isna()]['name'])
large=len(descanso.name)


if (large)>1:
    for i in range(large):
        paste_df= data[data['name'] == descanso.iloc[i,0]]
        paste_df['Days']=pd.to_datetime(paste_df['Days'], errors='coerce',format='%Y-%m-%d')
        dataframe=dataframe.append(paste_df, ignore_index = True)
        
elif large==1:
    paste_df=data[data['name'] == descanso.iloc[0,0]]
    paste_df['Days']=pd.to_datetime(paste_df['Days'], errors='coerce',format='%Y-%m-%d')
    dataframe=dataframe.append(paste_df, ignore_index = True)
else:
    pass

In [17]:
dataframe=dataframe.rename({'Days':'fecha','id_x':'workscheduleid','id_y':'nurseid'}, axis=1)
dataframe=dataframe[['fecha','nurseid','workscheduleid']] 
dataframe['fecha'] = dataframe['fecha'].dt.date


In [18]:
writer = pd.ExcelWriter('formato_homologacion_test.xlsx', engine='xlsxwriter')
dataframe.to_excel(writer, index_label=False, sheet_name='data', na_rep='', index=False)
writer.save()

In [19]:
df_inicial = pd.read_excel("./formato_homologacion_test.xlsx" , sheet_name='data', engine='openpyxl')